# Load packages

In [8]:
%load_ext autoreload
%autoreload 2
from covid_constants_and_util import *
import helper_methods_for_aggregate_data_analysis as helper
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
import datetime
import scipy
import time
import glob

JUST_TESTING = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Understand the Safegraph data from API

In [9]:
csv_dir = r'/media/gpu/Seagate/SC_weekly_patterns_20220101'

csv_parts = glob.glob(os.path.join(csv_dir, "SC-part*.csv.gz"))

csv_parts

['/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part1.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part2.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part3.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part4.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part5.csv.gz',
 '/media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part6.csv.gz']

In [ ]:
df = pd.read_csv(csv_parts[0])

df

## split according to the cbg

In [32]:
df['county_fips'] = df['sg_wp__poi_cbg'].astype(str).str.zfill(5).str[:6]
df['state_fips'] = df['sg_wp__poi_cbg'].astype(str).str.zfill(5).str[:2]
df['date_range'] = df['date_range_start'] + "_" + df['date_range_end']


In [34]:
df_gb = df.groupby(['state_fips', 'county_fips', 'date_range'])

In [56]:
def split_a_part(df, save_path, is_append=False, is_compressed=False):    
    suffix = 'csv'
    if is_compressed:
        suffix = 'csv.gz'

    cnt = 0
    for i, x in df_gb:
        # get data
        state_fips = x.iloc[0]['state_fips']
        county_fips = x.iloc[0]['county_fips']
        date_range = x.iloc[0]['date_range']

        # generate new file name
        folder = os.path.join(save_path, state_fips, county_fips)
        os.makedirs(folder, exist_ok=True)
        base_name = f"{county_fips}_{date_range}.{suffix}"
        new_name = os.path.join(folder, base_name)    

        if is_append:

            if not os.path.exists(new_name):
                x.to_csv(new_name, index=False)
            else:
                x.to_csv(new_name, index=False, mode='a', header=None)    
        else:
            x.to_csv(new_name, index=False)        

        x.to_csv()
        cnt += 1
        if cnt % 500 == 0:
            print(cnt, i, folder)
            
def split_all_part(csv_parts, save_path, is_append=False, is_compressed=False):
    for idx, csv_part in enumerate(csv_parts):
        print(f"\nProcessing {idx + 1}/{len(csv_parts)} file: {csv_part} \n")
        df = pd.read_csv(csv_part)
        split_a_part(df, save_path=save_path, is_append=is_append, is_compressed=is_compressed)d

save_path = r'/media/gpu/Seagate/SC_weekly_patterns_2020_split'  
csv_dir = r'/media/gpu/Seagate/SC_weekly_patterns_20220101'
csv_parts = glob.glob(os.path.join(csv_dir, "SC-part*.csv.gz"))

split_all_part(csv_parts, save_path=save_path, is_append=True, is_compressed=True)

Processing 1/6 file: /media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part1.csv.gz 

500 ('45', '450130', '2020-10-12_2020-10-19') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450130
1000 ('45', '450259', '2020-03-09_2020-03-16') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450259
1500 ('45', '450379', '2021-03-22_2021-03-29') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450379
2000 ('45', '450499', '2021-03-01_2021-03-08') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450499
2500 ('45', '450619', '2020-06-15_2020-06-22') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450619
3000 ('45', '450719', '2021-11-08_2021-11-15') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450719
3500 ('45', '450850', '2020-07-20_2020-07-27') /media/gpu/Seagate/SC_weekly_patterns_2020_split/45/450850
Processing 2/6 file: /media/gpu/Seagate/SC_weekly_patterns_20220101/SC-part2.csv.gz 

500 ('45', '450130', '2020-10-12_2020-10-19') /media/gpu/Seagate/SC_weekly_patterns_202

In [71]:
COLS_FROM_CORE_PLACES = ['safegraph_place_id', 'parent_safegraph_place_id', 'location_name', 'safegraph_brand_ids', 
                         'brands', 'top_category', 'sub_category', 'naics_code', 'latitude', 'longitude', 'street_address',
                         'city', 'region', 'postal_code', 'open_hours']

COLS_FROM_CORE_PLACES_FOOTPRINT = ['placekey', 'safegraph_place_id', 'polygon_class', 'area_square_feet', 'min_area']

COLS_FROM_WEEKLY_PATTERNS = ['sg_wp__parent_placekey', 'sg_wp__poi_cbg', 'sg_wp__visitor_home_cbgs', 'sg_wp__visitor_daytime_cbgs', 
                             'sg_wp__visitor_country_of_origin', 'sg_wp__distance_from_home', 'sg_wp__median_dwell', 'sg_wp__bucketed_dwell_times']

## Load the core_POI

In [62]:
core_poi_df = helper.load_core_places_data(COLS_FROM_CORE_PLACES)

Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part1.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part2.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part3.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part4.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part5.csv.gz
Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/2021/06/05/00/core_poi-part6.csv.gz
Loading core places info for 6462532 POIs


## Load the area column to the CORE_POIs.

In [73]:
def load_core_places_footprint_data(cols_to_keep):
    FOOTPRINT_FILE = r'/media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/core_POI_area.csv' 
    area_csv = os.path.join(FOOTPRINT_FILE)
    print('Loading', area_csv)
    df = load_csv_possibly_with_dask(area_csv, usecols=cols_to_keep, use_dask=False)
    df = df.set_index('safegraph_place_id')
    print('Loaded core places footprint data for %d POIs' % len(df))
    return df

footprint_df = load_core_places_footprint_data(COLS_FROM_CORE_PLACES_FOOTPRINT)
# footprint_df = load_core_places_footprint_data(None)

Loading /media/gpu/easystore/Safegraph/Core Places US (Nov 2020 - Present)/core_poi/core_POI_area.csv
Loaded core places footprint data for 5941711 POIs


In [74]:
footprint_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5941711 entries, sg:000000c8f55d451ca1ae98bd057785a7 to sg:fffffc1c8810425e8197760fc5328e89
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   polygon_class     object 
 1   area_square_feet  float64
 2   placekey          object 
 3   min_area          float64
dtypes: float64(2), object(2)
memory usage: 226.7+ MB


In [75]:
footprint_df['area_square_feet'] = footprint_df['min_area']
footprint_df

,polygon_class,area_square_feet,placekey,min_area
safegraph_place_id,,,,
sg:000000c8f55d451ca1ae98bd057785a7,OWNED_POLYGON,239.658301,222-222@628-tbd-cwk,239.658301
sg:000008c003ab41d9b6a8d1e58e4992ea,OWNED_POLYGON,95.644273,228-222@63r-6bj-hnq,95.644273
sg:0000093d8e7741e8a2a7ab13d432aaa2,NaN,376.543914,226-223@627-ytj-5xq,376.543914
sg:00000a99a12a45cd80342b9f881eced6,NaN,1236.058087,zzw-222@63c-pg6-2ff,1236.058087
sg:00000b5b33aa4d27a98b815f163ccd3b,OWNED_POLYGON,181.531029,zzw-222@8fc-63c-m6k,181.531029
...,...,...,...,...
sg:fffff90467044c7789e1594fdae67934,NaN,204.502240,224-222@5z4-kz8-2ff,204.502240
sg:fffff977b3324b7eac27c39b3036ff0f,NaN,668.063234,22d-222@5t4-thq-rzf,668.063234
sg:fffffafcfc1246349f856586a4f92d74,OWNED_POLYGON,3358.956378,222-222@62k-r7r-5s5,3358.956378


# Combine Core/Footprint/Weekly data and write out to MSA files

In this section, we process the raw SafeGraph Weekly Patterns data and divide the processing output into separate csv files per metropolitan statistical area (MSA). In processing the data, we:
- Expand the visits_by_each_hour column into one column per hour.
- Correct the visit spikes at the GMT midnight boundary (this is a known error with hourly SafeGraph data).
- Drop parent POIs so we don't double-count visits.
- Add an MSA column based on the POI's census block group (if that information is provided).

## Set the start/end date

In [80]:
# first and last week of the latest Weekly data to process
start_date = datetime.datetime(2020, 6, 29)   # Should be the data release date, i.e., the csv file name
end_date = datetime.datetime(2020, 8, 10)
print('Time period: %s to %s (%s)' % (start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), end_date - start_date))

Time period: 2020-06-29 to 2020-08-10 (42 days, 0:00:00)


## Select POIs in the AOI

In [83]:
# Select the South Carolina 
core_poi_df = core_poi_df[core_poi_df['region'] == 'SC']
core_poi_df

,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours
safegraph_place_id,,,,,,,,,,,,,,
sg:0123236b8d4c4e81ac12673c3e621ef0,NaN,Toby's TNT Music,NaN,NaN,"Sporting Goods, Hobby, and Musical Instrument ...",Musical Instrument and Supplies Stores,451140.0,33.516832,-81.847263,2868 Augusta Rd,Langley,SC,29851,"{ ""Mon"": [[""9:30"", ""18:00""]], ""Tue"": [[""9:30"",..."
sg:08a7df7912ad4b5482b4a189f0bb02e6,NaN,S C Veterinary Internal Medicine,NaN,NaN,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,621111.0,34.062226,-81.133517,132 Stonemark Ln,Seven Oaks,SC,29210,NaN
sg:0ff4a57b4adb42438cd1d08f0626fbef,NaN,Southern Tire of Summerville,NaN,NaN,"Automotive Parts, Accessories, and Tire Stores",Automotive Parts and Accessories Stores,441310.0,32.985588,-80.176531,427 Old Trolley Rd,Summerville,SC,29485,"{ ""Mon"": [[""8:00"", ""17:00""]], ""Tue"": [[""8:00"",..."
sg:1013084d2ba849988ce408184cd577ef,NaN,ALDI,SG_BRAND_41267d9fb3826d417e32077a3b0a35e5,ALDI,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,35.052030,-80.991332,3665 Vandora Springs Rd,Fort Mill,SC,29708,"{ ""Mon"": [[""9:00"", ""20:00""]], ""Tue"": [[""9:00"",..."
sg:1aeaadadec3d48838d252ae30dc8f9cf,NaN,Franklin Baking,NaN,NaN,Bakeries and Tortilla Manufacturing,Retail Bakeries,311811.0,33.856068,-78.660151,4308 Sea Mountain Hwy,Little River,SC,29566,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sg:e5292fd98e4e373ea9423b1129479957,NaN,Swami Food Store,NaN,NaN,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,34.513375,-82.664592,830 Bleckley St,Anderson,SC,29625,NaN
sg:e5eee29d6c714f069b616024cb2922d4,NaN,Joseph Cipriano,NaN,NaN,Offices of Other Health Practitioners,Offices of Chiropractors,621310.0,34.949171,-81.935556,104 N Daniel Morgan Ave,Spartanburg,SC,29306,"{ ""Mon"": [[""11:00"", ""20:00""]], ""Tue"": [[""11:00..."
sg:e7ea815079c9495eaae8f8d24793cc1e,sg:4bbf7541d80d49988629ed81831f3270,Western Union,SG_BRAND_9ee39f394d21a7f4848ab78a78da00c3,Western Union,Activities Related to Credit Intermediation,Other Activities Related to Credit Intermediation,522390.0,33.587628,-79.052726,11903 Highway 707,Murrells Inlet,SC,29576,NaN


In [88]:
# Remove parent POIs
parent_pois_id = core_poi_df.dropna(subset=['parent_safegraph_place_id'])['parent_safegraph_place_id'].unique()
print(f"Found {len(parent_pois_id)} parent POIs.")
parent_idx = core_poi_df['parent_safegraph_place_id'].isin(parent_pois_id)
core_poi_df = core_poi_df[~parent_idx]

core_poi_df

Found 0 parent POIs.


,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours
safegraph_place_id,,,,,,,,,,,,,,
sg:0123236b8d4c4e81ac12673c3e621ef0,NaN,Toby's TNT Music,NaN,NaN,"Sporting Goods, Hobby, and Musical Instrument ...",Musical Instrument and Supplies Stores,451140.0,33.516832,-81.847263,2868 Augusta Rd,Langley,SC,29851,"{ ""Mon"": [[""9:30"", ""18:00""]], ""Tue"": [[""9:30"",..."
sg:08a7df7912ad4b5482b4a189f0bb02e6,NaN,S C Veterinary Internal Medicine,NaN,NaN,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,621111.0,34.062226,-81.133517,132 Stonemark Ln,Seven Oaks,SC,29210,NaN
sg:0ff4a57b4adb42438cd1d08f0626fbef,NaN,Southern Tire of Summerville,NaN,NaN,"Automotive Parts, Accessories, and Tire Stores",Automotive Parts and Accessories Stores,441310.0,32.985588,-80.176531,427 Old Trolley Rd,Summerville,SC,29485,"{ ""Mon"": [[""8:00"", ""17:00""]], ""Tue"": [[""8:00"",..."
sg:1013084d2ba849988ce408184cd577ef,NaN,ALDI,SG_BRAND_41267d9fb3826d417e32077a3b0a35e5,ALDI,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,35.052030,-80.991332,3665 Vandora Springs Rd,Fort Mill,SC,29708,"{ ""Mon"": [[""9:00"", ""20:00""]], ""Tue"": [[""9:00"",..."
sg:1aeaadadec3d48838d252ae30dc8f9cf,NaN,Franklin Baking,NaN,NaN,Bakeries and Tortilla Manufacturing,Retail Bakeries,311811.0,33.856068,-78.660151,4308 Sea Mountain Hwy,Little River,SC,29566,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sg:e24cb2758be74f309574ee510b1690d5,NaN,Skeeter's Southern BBQ,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,32.428663,-80.774718,17 Market,Beaufort,SC,29906,"{ ""Mon"": [], ""Tue"": [[""11:00"", ""21:00""]], ""Wed..."
sg:e5292fd98e4e373ea9423b1129479957,NaN,Swami Food Store,NaN,NaN,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,34.513375,-82.664592,830 Bleckley St,Anderson,SC,29625,NaN
sg:e5eee29d6c714f069b616024cb2922d4,NaN,Joseph Cipriano,NaN,NaN,Offices of Other Health Practitioners,Offices of Chiropractors,621310.0,34.949171,-81.935556,104 N Daniel Morgan Ave,Spartanburg,SC,29306,"{ ""Mon"": [[""11:00"", ""20:00""]], ""Tue"": [[""11:00..."


## Merge core_poi and area

In [89]:
# footprint_df = helper.load_core_places_footprint_data(COLS_FROM_CORE_PLACES_FOOTPRINT)
ids_in_footprint_but_not_core = set(footprint_df.index) - set(core_poi_df.index)
print(f"footprint_df row count: {len(footprint_df)}, core_POI row count: {len(core_poi_df)}")
print("Warning: %i/%i (%.1f%%) POIs in footprint but not core data; dropping these" % 
      (len(ids_in_footprint_but_not_core), len(footprint_df), 100. * len(ids_in_footprint_but_not_core) / len(footprint_df)))
core_poi_df = pd.merge(core_poi_df, footprint_df, how='left', left_index=True, right_index=True, validate='one_to_one')
print("Missing data for footprint columns")
print(pd.isnull(core_poi_df[footprint_df.columns]).mean())

footprint_df row count: 5941711, core_POI row count: 85446
Missing data for footprint columns
polygon_class       0.229174
area_square_feet    0.090338
placekey            0.090338
min_area            0.090338
dtype: float64


In [90]:
core_poi_df = core_poi_df.dropna(subset=['area_square_feet'])

In [92]:
core_poi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77727 entries, sg:0123236b8d4c4e81ac12673c3e621ef0 to sg:fbc4ee66a65245b6a0bec5e376a7d2d5
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   parent_safegraph_place_id  0 non-null      object 
 1   location_name              77727 non-null  object 
 2   safegraph_brand_ids        16895 non-null  object 
 3   brands                     16895 non-null  object 
 4   top_category               77408 non-null  object 
 5   sub_category               76721 non-null  object 
 6   naics_code                 77408 non-null  float64
 7   latitude                   77727 non-null  float64
 8   longitude                  77727 non-null  float64
 9   street_address             77727 non-null  object 
 10  city                       77727 non-null  object 
 11  region                     77727 non-null  object 
 12  postal_code                77727 non-null  

In [94]:
def get_all_files(root_dir, extions=[".gz"]):
    found_files = []
    for rt_dir, dirs, files in os.walk(root_dir):
        for ext in extions:
            ext = ext.lower()
            ext_len = len(ext)
            for file in files:
                file_ext = file[-(ext_len):]
                # print(file)
                file_ext = file_ext.lower()
                if file_ext == ext:
                    file_name = os.path.join(rt_dir, file)
                    found_files.append(file_name)

    return found_files


def get_dir_from_files(files):
    dirs = [os.path.dirname(f) for f in files]
    dirs = list(set(dirs))
    return dirs

def get_week_str(csv_dir):
    files = get_all_files(csv_dir, extions=[".gz"])
    dirs = get_dir_from_files(files)
    dirs = sorted(dirs)
    
    week_datetime_list = []
    for d in dirs:
        dir_name = os.path.dirname(d)
        week_str = dir_name[-10:].replace("/", "-")
        week_datetime = datetime.datetime.fromisoformat(week_str)
        week_datetime_list.append(week_datetime)
                          
    return week_datetime_list
                          
# csv_dir = r'/media/gpu/easystore/Safegraph/Weekly Places Patterns (for data from 2020-11-30 to Present)/patterns'
csv_dir = r'/media/gpu/easystore/Safegraph/Weekly Places Patterns Backfill for Dec 2020 and Onward Release/patterns_backfill/2020/12/14/21'
week_datetime_list = get_week_str(csv_dir)
week_datetime_list

ValueError: Invalid isoformat string: '21-2018-01'

In [ ]:
base = core_poi_df.copy()
week_datetime_list = get_week_str(csv_dir)

start_date = week_datetime_list.pop(0)

while start_date <= end_date:
    week_string = start_date.strftime('%Y-%m-%d')
    print('===== WEEK STRING: %s =====' % week_string)
    weekly_df = helper.load_weekly_patterns_v2_data(week_string, COLS_FROM_WEEKLY_PATTERNS, expand_hourly_visits=True)
    
    # COLS_FROM_WEEKLY_PATTERNS = ['safegraph_place_id', 'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs', 
    #                          'visitor_country_of_origin', 'distance_from_home', 'median_dwell', 'bucketed_dwell_times']
    # safegraph_place_id will be set to index, remaining 7 columns, hourly columns=168, total added columns= 7 + 168 = 175
    
    # add the releasing date in the column
    weekly_df = weekly_df.rename(columns={k:f'{week_string}.{k}' for k in weekly_df.columns if not(k.startswith('hourly_visits_'))})
    ids_in_weekly_but_not_core = set(weekly_df.index) - set(base.index)
    print("Warning: %i/%i (%.1f%%) POIs in weekly but not core data; dropping these" % 
          (len(ids_in_weekly_but_not_core), len(weekly_df), 100. * len(ids_in_weekly_but_not_core) / len(weekly_df)))
    
    # left merge means we are only keeping the POIs that we have core info for
    # base = pd.merge(base, weekly_df, how='left', left_index=True, right_index=True, validate='one_to_one')
    # 2021/05/26: MergeError: Merge keys are not unique in right dataset; not a one-to-one merge
    
    old_column_cnt = len(base.columns) # Error: field larger than field limit (131072). The following code aims to test this error.
    old_row_cnt = len(base)
    
    base = pd.merge(base, weekly_df, how='left', left_index=True, right_index=True, validate='one_to_one')
    
    new_column_cnt = len(base.columns) 
    new_row_cnt = len(base)
    
    print(f"New column count = {new_column_cnt}, old column count = {old_column_cnt}, adding {new_column_cnt - old_column_cnt} columns.")
    print(f"New row count = {new_row_cnt}, old column count = {old_row_cnt}, adding {new_row_cnt - old_row_cnt} rows.")
    
    # sprint("Missing data for weekly columns")
    # sprint(pd.isnull(base[weekly_df.columns]).mean())
    # start_date = start_date + datetime.timedelta(days=7)
    start_date = week_datetime_list.pop(0)
    print()